# Regression with an Insurance Dataset
## Playground Series - Season 4, Episode 12
Submissions are evaluated using the Root Mean Squared Logarithmic Error (RMSLE).

In [17]:
from google.colab import drive
import sys

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)
sys.path.append('/content/drive/MyDrive')
import utils


Mounted at /content/drive


In [18]:
import pandas as pd
import numpy as np
from utils import *
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import make_scorer
!pip install scikit-optimize
!pip install shap
from skopt import BayesSearchCV
from skopt.space import Real, Integer
import xgboost as xgb
from sklearn.model_selection import KFold
import shap

## Load Data

In [76]:
train_set = pd.read_csv('/content/drive/MyDrive/Playground Series - Season 4, Episode 12/data/train.csv')
test_set = pd.read_csv('/content/drive/MyDrive/Playground Series - Season 4, Episode 12/data/test.csv')

train_set.head()

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
0,0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,...,2.0,17.0,372.0,5.0,2023-12-23 15:21:39.134960,Poor,No,Weekly,House,2869.0
1,1,39.0,Female,31678.0,Divorced,3.0,Master's,NaN,15.569731,Rural,...,1.0,12.0,694.0,2.0,2023-06-12 15:21:39.111551,Average,Yes,Monthly,House,1483.0
2,2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,...,1.0,14.0,NaN,3.0,2023-09-30 15:21:39.221386,Good,Yes,Weekly,House,567.0
3,3,21.0,Male,141855.0,Married,2.0,Bachelor's,NaN,10.938144,Rural,...,1.0,0.0,367.0,1.0,2024-06-12 15:21:39.226954,Poor,Yes,Daily,Apartment,765.0
4,4,21.0,Male,39651.0,Single,1.0,Bachelor's,Self-Employed,20.376094,Rural,...,0.0,8.0,598.0,4.0,2021-12-01 15:21:39.252145,Poor,Yes,Weekly,House,2022.0


In [77]:
train_set.describe()

,id,Age,Annual Income,Number of Dependents,Health Score,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Premium Amount
count,1.200000e+06,1.181295e+06,1.155051e+06,1.090328e+06,1.125924e+06,835971.000000,1.199994e+06,1.062118e+06,1.199999e+06,1.200000e+06
mean,5.999995e+05,4.114556e+01,3.274522e+04,2.009934e+00,2.561391e+01,1.002689,9.569889e+00,5.929244e+02,5.018219e+00,1.102545e+03
std,3.464103e+05,1.353995e+01,3.217951e+04,1.417338e+00,1.220346e+01,0.982840,5.776189e+00,1.499819e+02,2.594331e+00,8.649989e+02
min,0.000000e+00,1.800000e+01,1.000000e+00,0.000000e+00,2.012237e+00,0.000000,0.000000e+00,3.000000e+02,1.000000e+00,2.000000e+01
25%,2.999998e+05,3.000000e+01,8.001000e+03,1.000000e+00,1.591896e+01,0.000000,5.000000e+00,4.680000e+02,3.000000e+00,5.140000e+02
50%,5.999995e+05,4.100000e+01,2.391100e+04,2.000000e+00,2.457865e+01,1.000000,1.000000e+01,5.950000e+02,5.000000e+00,8.720000e+02
75%,8.999992e+05,5.300000e+01,4.463400e+04,3.000000e+00,3.452721e+01,2.000000,1.500000e+01,7.210000e+02,7.000000e+00,1.509000e+03
max,1.199999e+06,6.400000e+01,1.499970e+05,4.000000e+00,5.897591e+01,9.000000,1.900000e+01,8.490000e+02,9.000000e+00,4.999000e+03


In [78]:
for col in train_set.columns:
  if train_set[col].dtype == 'object':
    print(col, train_set[col].unique())

for col in train_set.columns:
    print(col, train_set[col].isna().sum())

for col in test_set.columns:
  if train_set[col].dtype == 'object':
    print(col, test_set[col].unique())

for col in test_set.columns:
    print(col, test_set[col].isna().sum())

Gender ['Female' 'Male']
Marital Status ['Married' 'Divorced' 'Single' nan]
Education Level ["Bachelor's" "Master's" 'High School' 'PhD']
Occupation ['Self-Employed' nan 'Employed' 'Unemployed']
Location ['Urban' 'Rural' 'Suburban']
Policy Type ['Premium' 'Comprehensive' 'Basic']
Policy Start Date ['2023-12-23 15:21:39.134960' '2023-06-12 15:21:39.111551'
 '2023-09-30 15:21:39.221386' ... '2021-04-28 15:21:39.129190'
 '2019-11-14 15:21:39.201446' '2020-10-19 15:21:39.118178']
Customer Feedback ['Poor' 'Average' 'Good' nan]
Smoking Status ['No' 'Yes']
Exercise Frequency ['Weekly' 'Monthly' 'Daily' 'Rarely']
Property Type ['House' 'Apartment' 'Condo']
id 0
Age 18705
Gender 0
Annual Income 44949
Marital Status 18529
Number of Dependents 109672
Education Level 0
Occupation 358075
Health Score 74076
Location 0
Policy Type 0
Previous Claims 364029
Vehicle Age 6
Credit Score 137882
Insurance Duration 1
Policy Start Date 0
Customer Feedback 77824
Smoking Status 0
Exercise Frequency 0
Property 

In [94]:
train_set = pd.read_csv('/content/drive/MyDrive/Playground Series - Season 4, Episode 12/data/train.csv')
test_set = pd.read_csv('/content/drive/MyDrive/Playground Series - Season 4, Episode 12/data/test.csv')

# Age
train_set['Age'] = train_set['Age']

def categorize_age(age):
    if age < 25:
        return '18-25'
    elif age < 40:
        return '25-40'
    elif age < 60:
        return '40-60'
    elif age >= 60:
        return '60+'
    else:
        return 'nan'


train_set['Age Cat'] = train_set['Age'].apply(categorize_age)
train_set = pd.get_dummies(train_set, columns=['Age Cat'], drop_first=False, dtype=float)
train_set['Age'] = train_set['Age'] / 100
train_set['Age'] = train_set['Age'].replace('nan', None).fillna(train_set['Age'].mode()[0])

# Gender
train_set['Gender'] = train_set['Gender'].replace({'Male': 0, 'Female': 1})

# Annual Income
train_set['Annual Income isna'] = train_set['Annual Income'].isna().astype(int)
train_set['Annual Income'] = train_set['Annual Income'].fillna(train_set['Annual Income'].mean())
train_set['Monthly Income'] = train_set['Annual Income'] / 12
train_set['Annual Income SQRT'] = np.sqrt(train_set['Annual Income'])
train_set['Annual Income Log'] = np.log(train_set['Annual Income'])
train_set['Annual Income SQ'] = train_set['Annual Income'] ** 2

# Marital Status
train_set['Marital Status'].fillna('nan', inplace=True)
train_set['Marital Status Int'] = train_set['Marital Status'].replace({'Married': 1, 'Divorced': 0.3, 'Single': 0.2, 'nan': 0})
train_set = pd.get_dummies(train_set, columns=['Marital Status'], drop_first=False, dtype=float)

# Number of Dependents
train_set['Number of Dependents isna'] = train_set['Number of Dependents'].isna().astype(int)
train_set['Number of Dependents'] = train_set['Number of Dependents'].fillna(0)
train_set['Number of Dependents'] = train_set['Number of Dependents'] / train_set['Number of Dependents'].max()

# Education Level
train_set['Education Level'] = train_set['Education Level'].fillna('nan')
train_set['Education'] = train_set['Education Level'].copy()
train_set = pd.get_dummies(train_set, columns=['Education'], drop_first=False, dtype=float)
train_set['Education Level'] = train_set['Education Level'].replace({'nan': -1, 'High School': 0, "Bachelor's": 1, "Master's": 2, 'PhD': 3})

# Occupation
train_set['Occupation'] = train_set['Occupation'].fillna('nan')
train_set = pd.get_dummies(train_set, columns=['Occupation'], drop_first=False, dtype=float)

# Health Score
train_set['Health Score'] = train_set['Health Score'].fillna(-100)
train_set['Health Score'] = train_set['Health Score'] / 100

# Location
train_set = pd.get_dummies(train_set, columns=['Location'], drop_first=False, dtype=float)

# Policy Type
train_set = pd.get_dummies(train_set, columns=['Policy Type'], drop_first=False, dtype=float)

# Previous Claims
train_set['Previous Claims Isna'] = train_set['Previous Claims'].isna().astype(int)
train_set['Previous Claims'] = train_set['Previous Claims'].fillna(-100)

# Vehicle Age
train_set['Vehicle Age'] = train_set['Vehicle Age'].fillna(train_set['Vehicle Age'].mode()[0])

# Policy Start Date
train_set['Policy Start Date'] = pd.to_datetime(train_set['Policy Start Date'])
train_set['Policy Start Date Year'] = train_set['Policy Start Date'].dt.year
train_set['Policy Start Date Month'] = train_set['Policy Start Date'].dt.month
train_set['Policy Start Date Day'] = train_set['Policy Start Date'].dt.day
train_set['Policy Start Date Weekday'] = train_set['Policy Start Date'].dt.weekday

# Smocking Status
train_set['Smoking Status'] = train_set['Smoking Status'].replace({'Yes': 1, 'No': 0})

# Exercise Frequency
train_set['Exercise Frequency'] = train_set['Exercise Frequency'].replace({'Daily': 1, 'Weekly': 0.5, 'Monthly': 0.3, 'Rarely': 0})

# Property Type
train_set['Property Type Int'] = train_set['Property Type'].replace({'Condo': 1, 'House': 0.4, 'Apartment': 0.2})
train_set = pd.get_dummies(train_set, columns=['Property Type'], drop_first=False, dtype=float)

# Customer Feedback
train_set['Customer Feedback'] = train_set['Customer Feedback'].fillna('nan')
train_set = pd.get_dummies(train_set, columns=['Customer Feedback'], drop_first=False, dtype=float)

# Credit Score
train_set['Credit Score isna'] = train_set['Credit Score'].isna().astype(int)
train_set['Credit Score'] = train_set['Credit Score'].fillna(train_set['Credit Score'].mean())

# Insurance Duration
train_set['Insurance Duration'] = train_set['Insurance Duration'].fillna(train_set['Insurance Duration'].mode()[0])

<ipython-input-94-198e39cde828>:26: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_set['Gender'] = train_set['Gender'].replace({'Male': 0, 'Female': 1})
<ipython-input-94-198e39cde828>:37: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_set['Marital Status'].fillna('nan', inplace=True)
<ipy

In [ ]:
train_set = pd.read_csv('/content/drive/MyDrive/Playground Series - Season 4, Episode 12/data/train.csv')
test_set = pd.read_csv('/content/drive/MyDrive/Playground Series - Season 4, Episode 12/data/test.csv')

# Age
train_set['Age'] = train_set['Age']

def categorize_age(age):
    if age < 25:
        return '18-25'
    elif age < 40:
        return '25-40'
    elif age < 60:
        return '40-60'
    elif age >= 60:
        return '60+'
    else:
        return 'nan'


train_set['Age Cat'] = train_set['Age'].apply(categorize_age)
train_set = pd.get_dummies(train_set, columns=['Age Cat'], drop_first=False, dtype=float)
train_set['Age'] = train_set['Age'] / 100
train_set['Age'] = train_set['Age'].replace('nan', None).fillna(train_set['Age'].mode()[0])

# Gender
train_set['Gender'] = train_set['Gender'].replace({'Male': 0, 'Female': 1})

# Annual Income
train_set['Annual Income isna'] = train_set['Annual Income'].isna().astype(int)
train_set['Annual Income'] = train_set['Annual Income'].fillna(train_set['Annual Income'].mean())
train_set['Monthly Income'] = train_set['Annual Income'] / 12
train_set['Annual Income SQRT'] = np.sqrt(train_set['Annual Income'])
train_set['Annual Income Log'] = np.log(train_set['Annual Income'])
train_set['Annual Income SQ'] = train_set['Annual Income'] ** 2

# Marital Status
train_set['Marital Status'].fillna('nan', inplace=True)
train_set['Marital Status Int'] = train_set['Marital Status'].replace({'Married': 1, 'Divorced': 0.3, 'Single': 0.2, 'nan': 0})
train_set = pd.get_dummies(train_set, columns=['Marital Status'], drop_first=False, dtype=float)

# Number of Dependents
train_set['Number of Dependents isna'] = train_set['Number of Dependents'].isna().astype(int)
train_set['Number of Dependents'] = train_set['Number of Dependents'].fillna(0)
train_set['Number of Dependents'] = train_set['Number of Dependents'] / train_set['Number of Dependents'].max()

# Education Level
train_set['Education Level'] = train_set['Education Level'].fillna('nan')
train_set['Education'] = train_set['Education Level'].copy()
train_set = pd.get_dummies(train_set, columns=['Education'], drop_first=False, dtype=float)
train_set['Education Level'] = train_set['Education Level'].replace({'nan': -1, 'High School': 0, "Bachelor's": 1, "Master's": 2, 'PhD': 3})

# Occupation
train_set['Occupation'] = train_set['Occupation'].fillna('nan')
train_set = pd.get_dummies(train_set, columns=['Occupation'], drop_first=False, dtype=float)

# Health Score
train_set['Health Score'] = train_set['Health Score'].fillna(-100)
train_set['Health Score'] = train_set['Health Score'] / 100

# Location
train_set = pd.get_dummies(train_set, columns=['Location'], drop_first=False, dtype=float)

# Policy Type
train_set = pd.get_dummies(train_set, columns=['Policy Type'], drop_first=False, dtype=float)

# Previous Claims
train_set['Previous Claims Isna'] = train_set['Previous Claims'].isna().astype(int)
train_set['Previous Claims'] = train_set['Previous Claims'].fillna(-100)

# Vehicle Age
train_set['Vehicle Age'] = train_set['Vehicle Age'].fillna(train_set['Vehicle Age'].mode()[0])

# Policy Start Date
train_set['Policy Start Date'] = pd.to_datetime(train_set['Policy Start Date'])
train_set['Policy Start Date Year'] = train_set['Policy Start Date'].dt.year
train_set['Policy Start Date Month'] = train_set['Policy Start Date'].dt.month
train_set['Policy Start Date Day'] = train_set['Policy Start Date'].dt.day
train_set['Policy Start Date Weekday'] = train_set['Policy Start Date'].dt.weekday

# Smocking Status
train_set['Smoking Status'] = train_set['Smoking Status'].replace({'Yes': 1, 'No': 0})

# Exercise Frequency
train_set['Exercise Frequency'] = train_set['Exercise Frequency'].replace({'Daily': 1, 'Weekly': 0.5, 'Monthly': 0.3, 'Rarely': 0})

# Property Type
train_set['Property Type Int'] = train_set['Property Type'].replace({'Condo': 1, 'House': 0.4, 'Apartment': 0.2})
train_set = pd.get_dummies(train_set, columns=['Property Type'], drop_first=False, dtype=float)

# Customer Feedback
train_set['Customer Feedback'] = train_set['Customer Feedback'].fillna('nan')
train_set = pd.get_dummies(train_set, columns=['Customer Feedback'], drop_first=False, dtype=float)

# Credit Score
train_set['Credit Score isna'] = train_set['Credit Score'].isna().astype(int)
train_set['Credit Score'] = train_set['Credit Score'].fillna(train_set['Credit Score'].mean())

# Insurance Duration
train_set['Insurance Duration'] = train_set['Insurance Duration'].fillna(train_set['Insurance Duration'].mode()[0])

In [95]:
for col in train_set.columns:
 print(col, train_set[col].isna().sum())

id 0
Age 0
Gender 0
Annual Income 0
Number of Dependents 0
Education Level 0
Health Score 0
Previous Claims 0
Vehicle Age 0
Credit Score 0
Insurance Duration 0
Policy Start Date 0
Smoking Status 0
Exercise Frequency 0
Premium Amount 0
Age Cat_18-25 0
Age Cat_25-40 0
Age Cat_40-60 0
Age Cat_60+ 0
Age Cat_nan 0
Annual Income isna 0
Monthly Income 0
Annual Income SQRT 0
Annual Income Log 0
Annual Income SQ 0
Marital Status Int 0
Marital Status_Divorced 0
Marital Status_Married 0
Marital Status_Single 0
Marital Status_nan 0
Number of Dependents isna 0
Education_Bachelor's 0
Education_High School 0
Education_Master's 0
Education_PhD 0
Occupation_Employed 0
Occupation_Self-Employed 0
Occupation_Unemployed 0
Occupation_nan 0
Location_Rural 0
Location_Suburban 0
Location_Urban 0
Policy Type_Basic 0
Policy Type_Comprehensive 0
Policy Type_Premium 0
Previous Claims Isna 0
Policy Start Date Year 0
Policy Start Date Month 0
Policy Start Date Day 0
Policy Start Date Weekday 0
Property Type Int 0
Pr

In [86]:
train_set.head()
train_set.to_csv('/content/drive/MyDrive/Playground Series - Season 4, Episode 12/data/train_processed.csv', index=False)